In [ ]:
### Importing some of the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##### Reading the datasets
location='/Users/chaitanya/Data_science/datasets_NYC/'
files = ["ap_2010.csv", "class_size.csv", "demographics.csv", \
 "graduation.csv", "hs_directory.csv", "math_test_results.csv", "sat_results.csv"]
###### Creating an empty dictionary to store the dataset
data={}
for f in range(len(files)):
    d=pd.read_csv(''.join([location,files[f]]))
    data[files[f].replace(".csv", "")] = d
    key_data=data.keys()
### Visualise the data sets
#for i in range(len(key_data)):
#    print ("\n")
#    print (key_data[i])
#    print ("\n")
#    print (data.values()[i].head(3))
print "+++++++++++++++++++++++++++++++++++"
#print key_data
#for index in range(len(key_data)):
#    print key_data[index]
#    print ("\n")
#    print data[key_data[index]].columns
#    print ("\n")
    
### Renaming the dbn column in hs_directory dataframe
#print data['hs_directory']['dbn'].head()
data['hs_directory'].rename(columns={'dbn':'DBN'},inplace=True)
#print data['hs_directory'].columns
#print data['hs_directory']['DBN'].head()
data["class_size"]["DBN"] = data["class_size"].apply(lambda x: "{0:02d}{1}".format(x["CSD"],x["SCHOOL CODE"]), axis=1)
data["class_size"]["DBN"]
data['class_size'].rename(columns={'GRADE ':'GRADE'},inplace=True)
#del data['hs_directory']['dbn']
#data['hs_directory']=data['hs_directory'].rename(columns({'dbn':'DBN'})
#data['hs_directory']['DBN']=data['hs_directory']['dbn']
#print data['hs_directory']['DBN'].head()



### Survey dataset

survey_75 = pd.read_csv(''.join([location,'masterfile11_d75_final.txt']), delimiter="\t", encoding='windows-1252')
survey_not75 = pd.read_csv(''.join([location,'masterfile11_gened_final.txt']), delimiter="\t", encoding='windows-1252')
survey_75['flag75']=True
survey_not75['flag75']=False
survey=pd.concat([survey_75,survey_not75],axis=0)
#survey.head()
# Removing extraneuous columns from the dataset

survey.rename(columns={'dbn':'DBN'},inplace=True)
survey_fields = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", \
                 "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11",\
                 "com_t_11", "eng_t_10", "aca_t_11", "saf_s_11", "com_s_11", \
                 "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
survey = survey.loc[:,survey_fields]
print survey.info()
data['survey']=survey 
#### Collecting relevant data 
#### Removing unnecessary columns from the dataset
grade_flag=data["class_size"]['GRADE']=='09-12'
prgm_flag=data["class_size"]['PROGRAM TYPE']=='GEN ED'
data["class_size"]= data["class_size"][grade_flag & prgm_flag]
class_size= data["class_size"].groupby(data["class_size"]['DBN']).mean()
class_size.reset_index(inplace=True)
data["class_size"]=class_size



#### Collecting relevant data  from the demographics dataset

#print data["demographics"].head()
recent_sch_year=data["demographics"].sort_values('schoolyear',ascending=False)['schoolyear'].head(1).values[0]
demograph_data= data["demographics"][data["demographics"]['schoolyear']==recent_sch_year]
data["demographics"]=demograph_data
mathsdata=data["math_test_results"]
#print mathsdata.Year
gradeflag=mathsdata.Grade=='8'
yearflag=mathsdata.Year==2011
mathsdata=mathsdata[yearflag & gradeflag]
data["math_test_results"]=mathsdata
#data["math_test_results"].head()


#### Collecting relevant data  from the graduation dataset
data["graduation"]=data["graduation"][data["graduation"]['Cohort']=='2006']
data["graduation"]=data["graduation"][data["graduation"]['Demographic']=='Total Cohort']
#data["graduation"].head()

#### Calculating the total sat score 
cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for c in cols:
    data["sat_results"][c] = pd.to_numeric(data["sat_results"][c],errors='ignore')

data['sat_results']['sat_score'] = data['sat_results'][cols[0]]\
+ data['sat_results'][cols[1]] + data['sat_results'][cols[2]]
#data['sat_results']['sat_score'].head(15)

#### Getting the location of each school 
data["hs_directory"]['lat'] = data["hs_directory"]['Location 1'].\
apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[0])
data["hs_directory"]['lon'] = data["hs_directory"]['Location 1'].\
apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[1])
for c in ['lat', 'lon']:
    data["hs_directory"][c] = pd.to_numeric(data["hs_directory"][c],errors='ignore')
#data["hs_directory"].head()    

##### The entire datasets


print "++++++++++++++++++++++++++++"
#for k,v in data.items():
#    print(k)
#    print"/n"
#    print(v.head(4))


data_names=data.keys()
full_data=[data[data_names[v]] for v in range (len(data_names))]

flat_data_names = [k for k,v in data.items()]
flat_data = [data[k] for k in flat_data_names]
full = flat_data[0]
for i, f in enumerate(flat_data[1:]):
    name = flat_data_names[i+1]
    print(name)
    print(len(f["DBN"]) - len(f["DBN"].unique()))
    join_type = "inner"
    if name in ["sat_results", "ap_2010", "graduation"]:
        join_type = "outer"
    if name not in ["math_test_results"]:
        full = full.merge(f, on="DBN", how=join_type)
        
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']   
for col in cols:
    full[col] = pd.to_numeric(full[col],errors='ignore')

full[cols] = full[cols].fillna(value=0)
full["school_dist"] = full["DBN"].apply(lambda x: x[:2])
##### The full dataset
full = full.fillna(full.mean())
##### Computing the correlations 
full['sat_score']=full['sat_score'].replace('sss', np.nan)
full=full.loc[full['sat_score'].notnull()]
full['total_enrollment'].plot()
plt.show()